<a href="https://colab.research.google.com/github/Hagar633/Machine-learning-/blob/main/audioanalysis_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gdown

FILE_ID = "1_bhPRAIbb-ulV_iYgMXV-e3tMQSzG3s_"
!gdown --id $FILE_ID -O my_audio.mp4


!apt-get -y install ffmpeg
!pip install -q openai-whisper
!pip install -q gdown

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 10.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [6]:
from time import time_ns
%pip install -q -U "google-genai>=1.0.0"
from google.colab import userdata
from google import genai
from datetime import time
from enum import Enum
from pydantic import BaseModel
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY_1')

client = genai.Client(api_key=GOOGLE_API_KEY)
MODEL_ID = "gemini-2.5-flash-lite" # @param ["gemini-2.5-flash-lite","gemini-2.0-flash","gemini-2.5-flash","gemini-2.5-pro"] {"allow-input":true, isTemplate: true}


class Instruction_type(str, Enum):
    encouragement = "encouragement"
    discussion = "discussion"
    correction = "correction"

class Instruction(BaseModel):
    instruction: str
    timestamp: time
    type: Instruction_type


!gdown --id 1CpgRlxocsUqQKsCyD0JEOQ014ovK1M-p -O test2.mp4
!ffmpeg -i test2.mp4 -q:a 0 -map a test2.mp3
your_audio_file = client.files.upload(file='test2.mp3')
file_status = client.files.get(name=your_audio_file.name)
print(file_status.state)


response = client.models.generate_content(
  model=MODEL_ID,
  contents=[
    your_audio_file
  ],
  config={
    "system_instruction": """ listen to an audio recording of a football match
where the coach is giving instructions and feedback to players.

From the audio, output a JSON object with the following structure:

{
  "summary": "Short description of the match and coach’s general comments",
  "players": [
    {
      "player": "Player’s name or number or position or what's heard from them",
      "instruction_count": "Total number of instructions given to this player",
      "instructions": [
        {
          "instruction": "The specific football related instruction or feedback given by the coach",
          "classification": "One from: correction, encouragement, discussion",
          "timestamp": "Time in the video when the instruction was given (e.g. '00:12:35')"
        }
      ]
    }
  ]
}""",
    "response_mime_type": "application/json",
    "response_schema": {
      "type": "object",
      "properties": {
        "summary": {"type": "string"},
        "players": {
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
              "player": {"type": "string"},
              "instruction_count": {"type": "integer"},
              "instructions": {
                "type": "array",
                "items": {
                  "type": "object",
                  "properties": {
                    "instruction": {"type": "string"},
                    "classification": {
                      "type": "string",
                      "enum": ["encouragement", "correction", "discussion"]
                    },
                    "timestamp": {"type": "string"}
                  },
                  "required": ["instruction", "classification", "timestamp"]
                }
              }
            },
            "required": ["player", "instruction_count", "instructions"]
          }
        }
      },
      "required": ["summary", "players"]
    },
  },
)

print(response.text)


/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1CpgRlxocsUqQKsCyD0JEOQ014ovK1M-p
To: /content/test2.mp4
100% 49.2M/49.2M [00:00<00:00, 80.3MB/s]
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-